# Advanced Employee Data Analysis With Python

-- Subquery: select employees who left and meet the first set of filters
SELECT *
FROM (
    SELECT *
    FROM employee_attrition
    WHERE Attrition = 'Yes'
      AND YearsAtCompany < 10
      AND MonthlyIncome BETWEEN 2000 AND 15000
      AND JobRole IN ('Sales Executive', 'Research Scientist', 'Laboratory Technician')
) AS filtered_initial
-- Additional filtering
WHERE Age BETWEEN 25 AND 45
  AND PerformanceRating >= 3;

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
numeric_columns = [
    'Age', 'DailyRate', 'DistanceFromHome', 'MonthlyIncome',
    'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears',
    'TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole',
    'YearsSinceLastPromotion', 'YearsWithCurrManager'
]

df_numeric = df[numeric_columns]


NameError: name 'df' is not defined

In [3]:
# Drop rows with missing values (if any)
df_numeric_clean = df_numeric.dropna()

# Compute correlation matrix using NumPy
correlation_matrix = df_numeric_clean.corr()


NameError: name 'df_numeric' is not defined

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    linewidths=0.5,
    square=True
)
plt.title('Correlation Heatmap of Numeric Employee Attributes')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the filtered dataset (replace path if necessary)
filtered_df = pd.read_csv('/mnt/data/filtered_employee_attrition.csv')

# Filter for specific job roles and departments
filtered_trend_df = filtered_df[
    (filtered_df['JobRole'].isin(['Sales Executive', 'Research Scientist', 'Laboratory Technician'])) &
    (filtered_df['Department'].isin(['Sales', 'Research & Development']))
]

# Group and aggregate data
grouped = filtered_trend_df.groupby(['YearsAtCompany', 'JobRole', 'Department']).agg(
    mean_income=('MonthlyIncome', 'mean'),
    std_income=('MonthlyIncome', 'std'),
    count=('MonthlyIncome', 'count')
).reset_index()

# Keep only groups with at least 5 employees
grouped_reliable = grouped[grouped['count'] >= 5]

# Set Seaborn style
sns.set(style="whitegrid")

# Create the line chart with error bars
plt.figure(figsize=(14, 8))

# Plot for each (JobRole, Department) group
for (job_role, dept), group_data in grouped_reliable.groupby(['JobRole', 'Department']):
    plt.errorbar(
        group_data['YearsAtCompany'],
        group_data['mean_income'],
        yerr=group_data['std_income'],
        label=f'{job_role} - {dept}',
        marker='o',
        capsize=5
    )

# Plot customization
plt.title('Trend of Monthly Income Over Years at Company by Job Role and Department')
plt.xlabel('Years at Company')
plt.ylabel('Average Monthly Income')
plt.legend(title='Job Role - Department')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load your filtered dataset (adjust path if needed)
filtered_df = pd.read_csv('filtered_employee_attrition.csv')

# Filter relevant columns and drop missing values
df_3d = filtered_df[['Age', 'TotalWorkingYears', 'MonthlyIncome']].dropna()

# Prepare variables
X1 = df_3d['Age']
X2 = df_3d['TotalWorkingYears']
Y = df_3d['MonthlyIncome']

# Create design matrix (Age, TotalWorkingYears, Intercept)
X = np.column_stack((X1, X2, np.ones_like(X1)))

# Perform multiple linear regression
coeffs, _, _, _ = np.linalg.lstsq(X, Y, rcond=None)
a, b, c = coeffs

# Create grid for Age and TotalWorkingYears
age_range = np.linspace(X1.min(), X1.max(), 20)
tw_range = np.linspace(X2.min(), X2.max(), 20)
age_grid, tw_grid = np.meshgrid(age_range, tw_range)

# Calculate regression plane
z_plane = a * age_grid + b * tw_grid + c

# Create 3D scatter plot
scatter = go.Scatter3d(
    x=X1,
    y=X2,
    z=Y,
    mode='markers',
    marker=dict(
        size=5,
        color=Y,
        colorscale='Viridis',
        colorbar=dict(title='MonthlyIncome'),
        opacity=0.8
    ),
    name='Employees'
)

# Add regression plane
plane = go.Surface(
    x=age_grid,
    y=tw_grid,
    z=z_plane,
    colorscale='Reds',
    opacity=0.6,
    showscale=False,
    name='Regression Plane'
)

# Layout customization
layout = go.Layout(
    title='3D Scatter Plot with Regression Plane',
    scene=dict(
        xaxis_title='Age',
        yaxis_title='Total Working Years',
        zaxis_title='Monthly Income'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

# Create and show the figure
fig = go.Figure(data=[scatter, plane], layout=layout)
fig.show()
